In [ ]:
import torch
import numpy as np


import torchvision
from torchvision import models

import torch.nn as nn

from skimage import io

import random
import time
import os.path


import matplotlib.pyplot as plt
from IPython.display import clear_output


from sklearn.metrics import accuracy_score
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


import torch.optim as optim
import torch.optim.lr_scheduler
import torch.nn.init
from torch.autograd import Variable


In [ ]:
import sys
sys.path.append('../')

In [ ]:
import src

In [ ]:
#src.make_dataloaders
#src.img_loader
#src.train
#src.Load_model
#src.validation

In [ ]:
def get_random_pos(img, window_shape = [55,55] ):
    """ Extract of 2D random patch of shape window_shape in the image """
    w, h = window_shape
    W, H = img.shape[-2:]
    x1 = random.randint(0, W - w - 1)
    y1 = random.randint(0, H - h - 1)

    return x1, x1 + w, y1, y1 + h #x1, x2, y1, y2

def random_crop_area(img):
    x1,x2,y1,y2 = get_random_pos(img)
    Sen_Im = img[:, x1:x2,y1:y2]
    return resize(Sen_Im,img.shape,anti_aliasing=True)



def sigmoid(z):
    z = np.clip(z, -100, np.inf)
    return 1 / (1 + np.exp(-z))


def cus_aug(data):
    data = torch.rot90(data,random.randint(-3,3), dims=random.choice([[3,2],[2,3]]))
    if random.random()>0.75:
        data = torch.flip(data, dims = random.choice([[2,],[3,],[2,3]]))
    pixmis = torch.empty_like(data).random_(data.shape[-1])
    pixmis = torch.where(pixmis>(data.shape[-1]/8),torch.ones_like(data),torch.zeros_like(data))
    return data* pixmis


def accuracy(gt_S,pred_S):       
    _, alp = torch.max(torch.from_numpy(pred_S), 1)
    return accuracy_score(gt_S,np.asarray(alp)) 

In [ ]:
data_folder = 'C:/dev/EuroSAT/'
root = os.path.join(data_folder, '2750/')

In [ ]:
lr = 0.001             
milestones = [50,75,90] 
epochs = 100            

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
if __name__ == '__main__':
    torch.manual_seed(0)
    np.random.seed(0)
    torch.cuda.manual_seed(0)
    random.seed(0)
    
    print("pytorch version", torch.__version__)
    
    criteria = torch.nn.CrossEntropyLoss()
    net = src.Load_model(resnet_val=18, device = device)
    train_loader, val_loader = src.make_dataloaders(root)
    
    net=net.to(device) # Move model to CPU
    
    optimizer = optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)


In [ ]:
net = src.train(net, train_loader, val_loader, criteria, optimizer, epochs, scheduler)
